This is the Bidirectional LSTM (RNN) based semantic section classiifer based on the classes from ontoogy.
We implmented this classifier based on word based cnn and using keras backened tensorflow.

In [1]:
import tensorflow as tf
from tensorflow.contrib import keras
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import datasets
from tensorflow.contrib.keras import layers
from tensorflow.contrib.keras import preprocessing
from tensorflow.contrib.keras import backend as K
from tensorflow.contrib.keras import callbacks
from tensorflow.contrib.keras import utils

from collections import defaultdict
import pandas as pd
import numpy as np
import os
import json
import pickle
from datetime import datetime
import sys
from sklearn import metrics,cross_validation
from tensorflow.contrib.keras.python.keras.layers.wrappers import Bidirectional
learn = tf.contrib.learn

tf.logging.set_verbosity(tf.logging.WARN)

/home/mahbubur/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.85
config.gpu_options.visible_device_list="0"
session = tf.Session(config=config)
keras.backend.set_session(session)
K.gpu_setup = ["gpu0"]

In [3]:
max_features = 0 # will be replace by the voc size after processing the input text
maxlen = 600
batch_size = 100
embedding_dims = 100
filters = 100
kernel_size = 3
hidden_dims = 100
epochs = 100
title_cat_aspects="data/Stratified_whole_semantic_section_classifier"
training_cat_aspects="data/Stratified_training_semantic_section_classifier"
validation_cat_aspects="data/Stratified_test_semantic_section_classifier"
max_count=999999999

model_dir="LSTM_models_semantic_higher_input_length_char/"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
embeddings_layer_names=None
model_name="lstm_semantic_embedding_ep"+str(epochs)+"_em_dim"+str(embedding_dims)
chunksize = 10 ** 5

sys.stdout = open(model_dir+'report_semantic_section_classifier_char'+ str(datetime.now()).replace(" ","-").replace(":","-")
                  +'.txt', 'w')

In [4]:
item_dataset = defaultdict(lambda : None)
item_dataset['target'] =[]   # target is the category id 
item_dataset['title'] =[]

label_cat_id_to_int={}

In [5]:
def process_chunks_to_get_input_data(data_from_file,item_dataset):
    for index, row in data_from_file.iterrows():
#         if row["title"] == "" or row["title"] =="title" or row.isnull().any():
#             row = row.fillna("nan")
        try:
            item_dataset['title'].append(row["title"].lower())
            # mapping category_id to int number for class labels
            if row["category"] in label_cat_id_to_int:
                item_dataset['target'].append(label_cat_id_to_int[row["category"]])
            else:
                label_cat_id_to_int[row["category"]]=len(label_cat_id_to_int)
                item_dataset['target'].append(label_cat_id_to_int[row["category"]])
                    
        except:
            """"""

def process_chunks_to_get_test_data(data_from_file,item_dataset_test):
    for index, row in data_from_file.iterrows():
        try:
            item_dataset_test['title'].append(row["title"].lower())
            item_dataset_test['target'].append(label_cat_id_to_int[row["category"]])
                    
        except:
            """"""



In [6]:
count_chunk=0
for chunk in pd.read_csv(title_cat_aspects, sep="\t", chunksize=chunksize):
    print "Processing chunk: ",count_chunk+1
    process_chunks_to_get_input_data(chunk,item_dataset)
    #if count_chunk >=2:
    #    break
    count_chunk+=1

In [7]:
max_count= len(item_dataset['target'])
print "total targets: ",max_count
print "unique targets: ", len(set(item_dataset['target']))


In [8]:
#processing 
print "Processing training data for voc generation"
x_train,y_train = item_dataset['title'],item_dataset['target']
del item_dataset # after getting training an test data delete the item_dataset to release memory.

with open(model_dir+'x_train_dump.json', 'w') as fp:
    json.dump(x_train, fp)

with open(model_dir+'label_cat_id_to_int.json', 'w') as fp:
    json.dump(label_cat_id_to_int, fp)


In [9]:
print "Processing voc"
char_processor = learn.preprocessing.ByteProcessor(maxlen)
x_train = np.array(list(char_processor.fit_transform(x_train)))
print "voc processing done!"
x_train.dump(model_dir+"x_train_np_dump.dat")
del x_train

pickle.dump(char_processor, open(model_dir+"char_processor",
                                 'wb'))

In [ ]:
# load vocab_processor
#char_processor = pickle.load(open(model_dir+"char_processor",'rb'))


In [10]:
max_char = 256

In [ ]:
#label_cat_id_to_int = json.load(open(model_dir+"label_cat_id_to_int.json","r"))

In [11]:
print('Build model...')

input_title =  layers.Input(shape=(maxlen,), name = 'input_title')

embedding_title= layers.Embedding(max_char,embedding_dims,
                    input_length=maxlen)(input_title)

bid_lstm1= Bidirectional(layers.LSTM(embedding_dims, dropout=0.2, recurrent_dropout=0.2,return_sequences=True
                                           ))(embedding_title)
bid_lstm2 = Bidirectional(layers.LSTM(embedding_dims))(bid_lstm1)

main_output = layers.Dense(len(label_cat_id_to_int), activation='softmax', name='main_output')(bid_lstm2)

model = models.Model(inputs=[input_title], outputs=[main_output])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())



In [12]:
tb_callback = callbacks.TensorBoard(log_dir=model_dir, histogram_freq=50, write_graph=True,
                          #embeddings_freq=50,
                          embeddings_layer_names=None)




In [13]:
filepath=model_dir+"weights-improvement-{epoch:02d}-{val_acc:.9f}.hdf5"
checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max'
                                      )
history = callbacks.History()


In [17]:
def generate_data_from_file(path, aspectTransformationDict):
    result = {}
    while True:
        for training_data in pd.read_csv(path, chunksize=128, sep='\t'):
            x = pd.DataFrame(list(char_processor.transform(map(lambda i:i.lower(),list(training_data['title'].fillna("nan")))))).as_matrix()
            y = [label_cat_id_to_int[i] for i in training_data['category']]
            y = keras.utils.to_categorical(y,num_classes=len(label_cat_id_to_int))
            yield x, y


In [18]:
model.fit_generator(generate_data_from_file(training_cat_aspects,label_cat_id_to_int),
                            validation_data=generate_data_from_file(validation_cat_aspects,label_cat_id_to_int),
                            steps_per_epoch=4608,
                            validation_steps=1,
                            epochs=epochs,callbacks=[tb_callback,checkpoint,history],
                            verbose=2)

In [19]:
model.save(model_dir+"final_model")

In [20]:
# process test data for p,r,f1-score calculation
item_dataset_test = defaultdict(lambda : None)
item_dataset_test['target'] =[]   # target is the category id 
item_dataset_test['title'] =[]

count_chunk=0
for chunk in pd.read_csv(validation_cat_aspects, sep="\t", chunksize=chunksize):
    print "Processing chunk: ",count_chunk+1
    process_chunks_to_get_test_data(chunk,item_dataset_test)
    #if count_chunk >=2:
    #    break
    count_chunk+=1



In [21]:
x_test = np.array(list(char_processor.transform(item_dataset_test['title'])))
x_test.dump(model_dir+"x_test_np_dump.dat")
y_test = keras.utils.to_categorical(item_dataset_test['target'],num_classes=len(label_cat_id_to_int))
pickle.dump(y_test, open(model_dir+"y_test",'wb'))


In [22]:
score = np.asarray(model.predict(x_test)).argmax(1)
predict = np.round(np.asarray(model.predict(x_test))).argmax(1)
targ = y_test.argmax(1)

print(metrics.classification_report(targ, predict))
print metrics.confusion_matrix(targ, predict)
